<a href="https://colab.research.google.com/github/deepanshu9012/Social_Media_Post_-_Caption_Generator_Using_GenAI/blob/main/Social_media_post_%26_caption_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Setup and Library Installation
This section installs and imports the necessary libraries for the project: `gradio` for the web interface and `transformers` for accessing pre-trained models.

In [ ]:
!pip install transformers gradio

In [ ]:
import gradio as gr
from transformers import pipeline
import random
import torch

### 2. Model and Pipeline Initialization
Here, we initialize the core components:
- **`caption_generator`**: A text-generation pipeline using the `gpt2` model to create captions.
- **`emoji_dict`**: A dictionary that categorizes emojis by sentiment (positive, negative, neutral).
- **`sentiment_pipeline`**: A sentiment-analysis pipeline to determine the emotion of the generated caption.

In [ ]:
caption_generator = pipeline("text-generation", model="gpt2")

In [ ]:
emoji_dict = {
    "positive": ["😊", "😃", "😄", "😁", "😆", "😅", "😂", "🤣", "😍", "😘", "😗", "😙", "😚", "😋", "😜", "😝", "😛", "🤗", "🤩", "🥳", "😇", "🌟"],
    "negative": ["😞", "😟", "😠", "😡", "😔", "😕", "🙁", "☹️", "😣", "😖", "😫", "😩", "😢", "😭", "😤", "😮‍💨", "😱", "😨", "😰", "😥", "😓", "🫌"],
    "neutral":  ["😐", "😑", "😶", "😏", "😒", "🙄", "😬", "🤔", "🤨", "😳", "😞"]
}

sentiment_pipeline = pipeline("sentiment-analysis")

### 3. Helper Functions
These are the functions that generate specific parts of the social media post.
- **`get_emojis(text)`**: Analyzes the caption's sentiment and returns a random selection of three matching emojis.
- **`get_hashtags(prompt, platform)`**: Creates relevant hashtags from the user's prompt and adds popular, platform-specific tags.

In [ ]:
def get_emojis(text):
    label = sentiment_pipeline(text)[0]['label'].lower()
    return ' '.join(random.sample(emoji_dict.get(label, ["😶"]), 3))

def get_hashtags(prompt, platform):
    words = prompt.lower().split()
    tags = ["#" + word.replace(" ", "") for word in words if len(word) > 3]

    platform_tags = {
        "Instagram": ["#instadaily", "#igers", "#picoftheday"],
        "LinkedIn": ["#career", "#leadership", "#networking"],
        "Twitter": ["#tweet", "#trending", "#news"]
    }

    return " ".join(tags[:5] + random.sample(platform_tags[platform], 2))

### 4. Main Post Generation Function
The `generate_post` function orchestrates the entire process. It takes a user's prompt and chosen platform, then calls the caption, emoji, and hashtag generator functions to assemble the final post.


In [ ]:
def generate_post(prompt, platform):
    # Generate caption
    caption = caption_generator(prompt, max_length=50, num_return_sequences=1)[0]['generated_text']

    # Generate emojis
    emojis = get_emojis(caption)

    # Generate hashtags
    hashtags = get_hashtags(prompt, platform)

    return caption.strip(), emojis, hashtags

### 5. Gradio Web Interface
This final section builds and launches the user interface using the `gradio` library. It defines the input fields (a text box and radio buttons) and the output fields to display the generated caption, emojis, and hashtags.

In [ ]:
interface = gr.Interface(
    fn=generate_post,
    inputs=[
        gr.Textbox(label="Enter keyword or theme"),
        gr.Radio(["Instagram", "LinkedIn", "Twitter"], label="Choose Platform")
    ],
    outputs=[
        gr.Textbox(label="Generated Caption"),
        gr.Textbox(label="Emojis"),
        gr.Textbox(label="Hashtags")
    ],
    title="Social Media Post & Caption Generator",
    description="Generate catchy captions, relevant hashtags, and emojis based on your theme!"
)

interface.launch()